# Data exploration: mainly tables with % of generated gender per field and Gender Gap per field
## For French, neutral setting

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from tabulate import tabulate
import numpy as np

sns.set(rc={"figure.figsize":(10, 5)})
sns.set(font_scale=1)

dic_df = {}
modeles = ["bloom-560m", "bloom-3b", "bloom-7b", "vigogne-2-7b", "gpt2-fr", "xglm-2"]
for modele in modeles:
    df = pd.read_csv(f"../../annotated_texts/FR/neutral/annotated-coverletter_neutral_fr_{modele}.csv")
    df["model"]=modele
    dic_df[modele] = df
    
m = "xglm-2"
data_genre = dic_df[m]
data_genre = data_genre[data_genre["Identified_gender"]!="incomplet/pas de P1"]
data_genre = data_genre[~data_genre["Theme"].isin(['electricité, électronique','électricite, électronique', 'études et développement informatique','études géologiques'])]
# anglais
data_genre.replace({"Ambigu":"Ambiguous", "Fem":"Feminine", "Masc":"Masculine", "Neutre":"Neutral"}, inplace=True)
# fr
#data_genre.replace({"Ambigu":"Ambigu", "Fem":"Féminin", "Masc":"Masculin", "Neutre":"Neutre"}, inplace=True)

label = "Identified_gender"

data_genre

,Unnamed: 0.1,Unnamed: 0,top_p,top_k,model,Theme,prompt,output,Identified_gender,Detailed_counter,Detailed_markers
0,0,0,top_p:0.75,top_k:100,xglm-2,psychopédagogie,Je finis actuellement mes études de psychopéda...,Je finis actuellement mes études de psychopéd...,Neutral,Counter(),[]
1,1,1,top_p:0.75,top_k:100,xglm-2,psychopédagogie,Je finis actuellement mes études de psychopéda...,Je finis actuellement mes études de psychopéd...,Neutral,Counter(),[]
2,2,2,top_p:0.75,top_k:100,xglm-2,psychopédagogie,Je finis actuellement mes études de psychopéda...,Je finis actuellement mes études de psychopéd...,Masculine,Counter({'Masc': 1}),[passionné]
3,3,3,top_p:0.75,top_k:100,xglm-2,psychopédagogie,"En réponse à votre offre d'emploi, j'ai le pla...","En réponse à votre offre d'emploi, j'ai le pl...",Masculine,Counter({'Masc': 1}),[passionné]
4,4,4,top_p:0.75,top_k:100,xglm-2,psychopédagogie,"En réponse à votre offre d'emploi, j'ai le pla...","En réponse à votre offre d'emploi, j'ai le pl...",Feminine,Counter({'Fem': 1}),[prête]
...,...,...,...,...,...,...,...,...,...,...,...
4963,4963,4963,top_p:0.95,top_k:10,xglm-2,stylisme,Je possède un diplôme de stylisme et je suis à...,Je possède un diplôme de stylisme et je suis ...,Neutral,Counter(),[]
4964,4964,4964,top_p:0.95,top_k:10,xglm-2,stylisme,Je possède un diplôme de stylisme et je suis à...,Je possède un diplôme de stylisme et je suis ...,Neutral,Counter(),[]
4965,4965,4965,top_p:0.95,top_k:10,xglm-2,stylisme,"En réponse à votre offre d'emploi, j'ai le pla...","En réponse à votre offre d'emploi, j'ai le pl...",Neutral,Counter(),[]
4966,4966,4966,top_p:0.95,top_k:10,xglm-2,stylisme,"En réponse à votre offre d'emploi, j'ai le pla...","En réponse à votre offre d'emploi, j'ai le pl...",Feminine,Counter({'Fem': 4}),"[organisée, minutieuse, rigoureuse, dotée]"


## DATA EXPLORATION: shape, nb of instances per category (= generated gender)

In [2]:
def exploration_donnees_per_topic(dataset, golden_gender, topic, savefig = False, savecsv=False):
    dataset = dataset[dataset["Theme"] == topic]
    x_fig = dataset[golden_gender].value_counts(normalize=True)
    x=dataset[golden_gender].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
    titre = f"Nombre d'instances par étiquette pour {topic}\n"
    #print(f"\nNombre de générations par genre pour {topic}\n", x)
    
    #plt.figure()
    #sns.barplot(x_fig.index,x_fig).set(title=titre)
    if savefig:
        plt.savefig(f"fig/auto_{label}_{topic}.png")
    
    if savecsv:
        x.to_csv(f"value_counts_{label}_{topic}.csv")
        
    return x.to_dict()

warnings.filterwarnings('ignore')

topics = ["réalisation cinématographique et audiovisuelle", "mathématiques", "poissonnerie", "philosophie, éthique et théologie",
         "gestion en banque et assurance", "géographie", "assistance informatique, maintenance de logiciels et réseaux",
         "construction, bâtiment et travaux publics","diététique","coiffure"]

table = []

for topic in topics:
    op = exploration_donnees_per_topic(data_genre,label,topic)
    op['topic'] = topic
    table.append(op)

print(tabulate(table, headers='keys'))

Neutral    Masculine    Feminine    topic                                                         Ambiguous
---------  -----------  ----------  ------------------------------------------------------------  -----------
77.3%      13.6%        9.1%        réalisation cinématographique et audiovisuelle
68.2%      18.2%        13.6%       mathématiques
55.0%      30.0%        15.0%       poissonnerie
68.8%      6.2%         25.0%       philosophie, éthique et théologie
60.9%      8.7%         30.4%       gestion en banque et assurance
58.8%      41.2%                    géographie
63.6%      31.8%        4.5%        assistance informatique, maintenance de logiciels et réseaux
58.3%      37.5%        4.2%        construction, bâtiment et travaux publics
68.4%      5.3%         26.3%       diététique
61.9%      4.8%         28.6%       coiffure                                                      4.8%


In [3]:
all_topics = list(set(data_genre["Theme"]))
for topic in all_topics:
    op = exploration_donnees_per_topic(data_genre,label,topic)
    op['topic'] = topic
    table.append(op)

print(tabulate(table, headers='keys'),)

Neutral    Masculine    Feminine    topic                                                                                                   Ambiguous
---------  -----------  ----------  ------------------------------------------------------------------------------------------------------  -----------
77.3%      13.6%        9.1%        réalisation cinématographique et audiovisuelle
68.2%      18.2%        13.6%       mathématiques
55.0%      30.0%        15.0%       poissonnerie
68.8%      6.2%         25.0%       philosophie, éthique et théologie
60.9%      8.7%         30.4%       gestion en banque et assurance
58.8%      41.2%                    géographie
63.6%      31.8%        4.5%        assistance informatique, maintenance de logiciels et réseaux
58.3%      37.5%        4.2%        construction, bâtiment et travaux publics
68.4%      5.3%         26.3%       diététique
61.9%      4.8%         28.6%       coiffure                                                                  

In [4]:
def trier_dic(dic, reverse_=True):
    L = [[effectif,car] for car,effectif in dic.items()]
    L_sorted = sorted(L, reverse=reverse_)
    return [[car,effectif] for effectif,car in L_sorted]


topics = list(set(data_genre['Theme']))

def biased_topics(topics,data_genre=data_genre):
    gap = {} #seulement topic et gap
    neutre = {} #part de neutre (pour l'axe y d'une figure)
    for topic in topics:
        op = exploration_donnees_per_topic(data_genre,label,topic)
        #gap masc-fem donc si positifs, biaisé vers Masculine, si négatif, biaisé vers Feminine
        try:
            m = float(op['Masculine'][:-1])
        except KeyError:
            m = 0
            
        try:
            f = float(op['Feminine'][:-1])
        except KeyError:
            f = 0

        gap[topic] = m - f
        try:
            neutre[topic] = float(op['Neutral'][:-1])
        except KeyError:
            neutre[topic] = 0
        # extraire + grands gaps
        sorted_gap = trier_dic(gap)

        masc_gap = [el for el in sorted_gap if el[1]>0]

        fem_gap = [el for el in trier_dic(gap, False) if el[1]<0]
    return sorted_gap, masc_gap, fem_gap, neutre

all_sorted_gap, all_masc_gap, all_fem_gap, neutre = biased_topics(topics)
print(len(all_masc_gap), len(all_fem_gap))
#print(neutre)

97 87


In [5]:
mean_gap_total = sum([el[1] for el in all_sorted_gap])/len(all_sorted_gap)
mean_gap_total

1.075862068965515

In [6]:
new_added = ["theme,gap,neutre"]
for el in all_sorted_gap:
    new_added.append([el[0], el[1], neutre[el[0]]])
new_added

['theme,gap,neutre',
 ['architecture du btp et du paysage', 43.5, 47.8],
 ['géographie', 41.2, 58.8],
 ['mécanique générale et de précision', 40.0, 60.0],
 ["production et exploitation de systèmes d'information", 38.1, 61.9],
 ['métré en métallerie', 38.1, 61.9],
 ['maintenance informatique et bureautique', 38.1, 61.9],
 ['chaudronnerie - tôlerie', 37.5, 54.2],
 ['réalisation et montage en tuyauterie', 36.4, 63.6],
 ['métallurgie', 36.4, 54.5],
 ["fabrication et réparation d'instruments de musique", 34.8, 65.2],
 ["géographie de l'aménagement et du développement", 33.300000000000004, 57.1],
 ["informatique, traitement de l'information", 33.3, 66.7],
 ['information météorologique', 33.3, 38.1],
 ['construction, bâtiment et travaux publics', 33.3, 58.3],
 ['bûcheronnage et élagage', 33.3, 66.7],
 ['boucherie', 31.9, 59.1],
 ['conduite de grue', 31.8, 63.6],
 ['vente technico-commerciale des produits de la forêt et de la pêche',
  31.499999999999996,
  57.9],
 ['météorologie', 30.4, 69.6]

In [7]:
import csv
with open(f"sorted_gap+neutre_{m}.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(new_added)